In [2]:
import skopt
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.svm import SVC
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize

In [3]:
# Test!!

url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
df = pd.read_csv(url, header=None)
data = df.values

x, y = data[:, :-1], data[:, -1]

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEquidistantSelection(TransformerMixin, BaseEstimator):
    def __init__(self, n_features_to_select=None):
        self.n_features_to_select = n_features_to_select
        self.selected_features_idx = []

    def get_params(self, deep=True):
        return {"n_features_to_select": self.n_features_to_select}       

    def fit(self, X, y=None):
        n_features = X.shape[1]
        self.selected_features_idx = np.linspace(0, n_features-1, self.n_features_to_select, dtype=int)
        return self

    def transform(self, X, y=None, **kwargs):
        return X[:, self.selected_features_idx]

In [11]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([("transform", FeatureEquidistantSelection()), ('svc', SVC())])

{'memory': None,
 'steps': [('transform', FeatureEquidistantSelection()), ('svc', SVC())],
 'verbose': False,
 'transform': FeatureEquidistantSelection(),
 'svc': SVC(),
 'transform__n_features_to_select': None,
 'svc__C': 1.0,
 'svc__break_ties': False,
 'svc__cache_size': 200,
 'svc__class_weight': None,
 'svc__coef0': 0.0,
 'svc__decision_function_shape': 'ovr',
 'svc__degree': 3,
 'svc__gamma': 'scale',
 'svc__kernel': 'rbf',
 'svc__max_iter': -1,
 'svc__probability': False,
 'svc__random_state': None,
 'svc__shrinking': True,
 'svc__tol': 0.001,
 'svc__verbose': False}

In [4]:
from skopt import BayesSearchCV

# Define search space
params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=SVC(), search_spaces=params, n_jobs=-1, cv=cv)
# perform the search
search.fit(x, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

KeyboardInterrupt: 